In [2]:
import numpy as np
import pyomo
import pyomo.environ as pyo

# Pyomo

### Instantiate Model
The 

In [7]:
opt_model = pyo.ConcreteModel()

### Create Sets
These will be used to index constraints and variables

In [8]:
N = 48 #MPC HORIZON
NUMBER_OF_UNITS = 1


# Time Steps
opt_model.mpc_horizon = pyo.Set(initialize=range(0, N)) # set
# Units
opt_model.units_set = pyo.Set(initialize=range(NUMBER_OF_UNITS))

### Define Parameters
These are constant values and can be indexed over the defined sets

In [9]:
# Initialize Setpoint for t=0 to t=N
def setpoint_init(model, i, j): # functions always contain these input arguments: (model, set1_index, set2_index, etc..)
    return np.ones((N))[i]*21 # Assume setpoint is always 21 C

opt_model.setpoints = pyo.Param(opt_model.units_set, opt_model.mpc_horizon, # Index parameters over the sets [units, timestep]
                                initialize=setpoint_init, # Defines the values
                                mutable=True # Says it can be changed in the future (optional)
                               )

# Initialize Outdoor Temp for t=0 to t=N
def outdoor_temp_init(model, i, j): # functions always contain these input arguments: (model, set1_index, set2_index, etc..)
        return 21#outdoor_temp[j] # User defined weather object containing weather data

opt_model.outdoor_temp = pyo.Param(opt_model.units_set, opt_model.mpc_horizon, # Index parameters over the sets [units, timestep]
                                   initialize=outdoor_temp_init,
                                   mutable=True)

Tip on debugging/printing parameter values:

In [10]:
# This only prints the indexes of the parameters
print(list(opt_model.outdoor_temp))

# This calls and prints the actual parameter values for each index
for i in range(1,N):
    print(f'{pyo.value(opt_model.outdoor_temp[0, i]):.2f}', end=" ")

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), (0, 45), (0, 46), (0, 47)]
21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 21.00 

### Define Variables

In [11]:
# HP Control Variable
def u_hp_binary_init(model, i, j): # functions always contain these input arguments: (model, set1_index, set2_index, etc..)
    return 1
    
opt_model.u_hp_binary = pyo.Var(opt_model.units_set, opt_model.mpc_horizon, 
                                domain=pyo.Binary, #pyo.Reals,pyo.NonNegativeReals, etc.
                                initialize=u_hp_binary_init # This can be used for a warm start
                               )

# Indoor Temperature Variable
opt_model.indoor_temp = pyo.Var(opt_model.units_set, 
                                opt_model.mpc_horizon, 
                                domain=pyo.Reals,
                                bounds=(10, 35))
opt_model.mass_temp = pyo.Var(opt_model.units_set, 
                              opt_model.mpc_horizon, 
                              domain=pyo.Reals,
                                bounds=(10, 35))
# etc.

### Define Constraints

In [12]:
# Indoor Temperature Constraint
comfort_level = 1
def upper_temp_constraint(model, i, j): # constraints always contain these input arguments: (model, set1_index, set2_index, etc..)
    return model.indoor_temp[i, j]  <= model.setpoints[i, j] + comfort_level

opt_model.upper_temp_constraint = pyo.Constraint(opt_model.units_set, opt_model.mpc_horizon, # Sets to index over
                                                 rule=upper_temp_constraint # Function that defines the constraint
                                                )

### Define Objective Function

In [14]:
def obj(model): # Always pass in model. No need to index objective function since there is only one objective
    """
    Objective function for an aggregate of units

    :param opt_model:
    :return: cost: objective value
    """
    cost = 0
    for i in model.units_set:
        for j in model.mpc_horizon:
            cost = cost + model.u_hp_binary[i,j]
    return cost

opt_model.obj = pyo.Objective(rule=obj, sense=pyo.maximize)


### Solve

In [15]:
# Create your solver. This locates the executable used to solve the problem
solver = pyo.SolverFactory('cplex',
                        # executable=r'C:\Program Files\IBM\ILOG\CPLEX_Studio129\cplex\bin\x64_win64\cplex.exe'
                        )

# Solve the model
results = solver.solve(opt_model, 
                       tee=True, #Show log 
                       warmstart=True,
                       options={ # Options can be found on cplex website
                              'timelimit': 300,
                              # '1067': 36, #threads
                              #'mipgap': .1,
                              # '2010': .001 #optimality gap
                              }, 
                       keepfiles=True # Required if multiprocessing
                      )

# Just in case it's infeasible
if results.solver.termination_condition == pyomo.opt.TerminationCondition.infeasible:
    for i in opt_model.units_set:
        u_hp[i, 0] = pyo.value(opt_model.u_hp_binary[i, 0]) # If infeasible, initialize a control value
    print('INFEASIBLE!!!!')


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.9.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Zach\AppData\Local\Temp\tmp453h2h42.cplex.log' open.
CPLEX> New value for time limit in seconds: 300
CPLEX> Problem 'C:\Users\Zach\AppData\Local\Temp\tmpfkooqbih.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> MIP start file 'C:\Users\Zach\AppData\Local\Temp\tmpm713ct7n.cplex.mst' read.
CPLEX> Problem name         : C:\Users\Zach\AppData\Local\Temp\tmpfkooqbih.pyomo.lp
Objective sense      : Maximize
Variables            :      97  [Nneg: 1,  Box: 48,  Binary: 48]
Objective nonzeros   :      48
Linear constraints   :      49  [Less: 48,  Equal: 1]
  Nonzeros           :     

### Get Control Values

In [16]:
# Obtaining the decision variables from the model:
u_hp = np.zeros((len(opt_model.units_set), len(opt_model.mpc_horizon)))

for i in opt_model.units_set:
    for j in opt_model.mpc_horizon:
        u_hp[i, j] = pyo.value(opt_model.u_hp_binary[i, j])
        print(u_hp[i, j], end=" ")


1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 